# Extract generater model

Extrat only generater model from GAN model checkpoint

In [2]:
# Check target checkpoint
!gsutil ls gs://chair-generation/models/edges2chair_ikea/

gs://chair-generation/models/edges2chair_ikea/checkpoint
gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1565351093.tensorflow-20190806-153451
gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1565698804.tensorflow-20190806-153451
gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1565767058.tensorflow-20190806-153451
gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1566206506.chair-demo-instance
gs://chair-generation/models/edges2chair_ikea/graph.pbtxt
gs://chair-generation/models/edges2chair_ikea/model-315065.data-00000-of-00001
gs://chair-generation/models/edges2chair_ikea/model-315065.index
gs://chair-generation/models/edges2chair_ikea/model-315065.meta
gs://chair-generation/models/edges2chair_ikea/model-324165.data-00000-of-00001
gs://chair-generation/models/edges2chair_ikea/model-324165.index
gs://chair-generation/models/edges2chair_ikea/model-324165.meta
gs://chair-generation/models/edges2chair_ikea/model-344565.d

In [9]:
# Copy the model to local
!gsutil cp -r gs://chair-generation/models/edges2chair_ikea/ ../models

Copying gs://chair-generation/models/edges2chair_ikea/checkpoint...
Copying gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1565351093.tensorflow-20190806-153451...
Copying gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1565698804.tensorflow-20190806-153451...
Copying gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1565767058.tensorflow-20190806-153451...
| [4 files][261.4 MiB/261.4 MiB]   26.7 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1566206506.chair-demo-instance...
Copying gs://chair-generation/models/edges2chair_ikea/graph.pbtxt...            
Copying gs://chair-generation/models/edges2chair_ikea/model-31506

In [ ]:
# Export Generater Checkpoint and saved model
!python ../pix2pix.py \
  --mode export \
  --output_dir ../export/edges2chair_ikea/ \
  --checkpoint ../models/edges2chair_ikea/

In [ ]:
# Copy to GCS
!gsutil cp -r ../export/edges2chair_ikea/ gs://chair-generation/export

# Deploy model

In [13]:
!gcloud ai-platform models create "chair_generation"

Created ml engine model [projects/chair-search-demo/models/chair_generation].


In [2]:
MODEL_DIR="gs://chair-generation/export/edges2chair_ikea/export"
VERSION_NAME="v1"
MODEL_NAME="chair_generation"
FRAMEWORK="tensorflow"

!gcloud ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --origin $MODEL_DIR \
  --runtime-version=1.14 \
  --framework $FRAMEWORK \
  --python-version=3.5

Creating version (this might take a few minutes)......done.                    


# Test

Check [Test_saved_model.ipynb](Test_saved_model.ipynb) for JSON file creation

In [ ]:
""" Test model with gcloud command. Output with:
    OUTPUT_BYTES
    {u'b64': u"..."}
"""
!gcloud ai-platform predict --model $MODEL_NAME  \
                   --version $VERSION_NAME \
                   --json-instances instance.json

In [31]:
import googleapiclient 
from oauth2client.client import GoogleCredentials
import json

In [25]:
def predict_json(project, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    service = googleapiclient.discovery.build('ml', 'v1',)
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [ ]:
import base64

# Open and read image as bitstring
input_image = open("../data/sketch/cpvr2016/train/1.png", "rb").read()
print("Raw bitstring: " + str(input_image[:10]) + " ... " + str(input_image[-10:]))

# Encode image in b64
encoded_input_string = base64.b64encode(input_image)
input_string = encoded_input_string.decode()
request_data = {'image_bytes': {"b64": input_string}}

output = predict_json("chair-search-demo", "chair_generation", request_data, version=None)